# Atividade Prática: Sistema de Predição de Voto

##  Contexto

Vamos criar um sistema de inferência fuzzy para prever a **chance de um eleitor votar** em um candidato, baseado em:
- **Renda Familiar** (0 a 30 mil R$)
- **Escolaridade** (0 a 15 anos de estudo)

Este é um exemplo didático para praticar sistemas fuzzy com múltiplas entradas!

---

### Setup

In [ ]:
!pip install pyfuzzy-toolbox -q

print('✅ pyfuzzy-toolbox instalado!')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import fuzzy_systems as fs
from fuzzy_systems import MamdaniSystem

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 6)

print('✅ Bibliotecas carregadas!')
print(f'   pyfuzzy-toolbox: {fs.__version__}')

### Step 1

In [ ]:
# ============================================================================
# Criar Sistema Mamdani
# ============================================================================

sistema = MamdaniSystem()

# ============================================================================
# Variável 1: Renda Familiar
# ============================================================================

sistema.add_input('renda', (0, 30))

sistema.add_term('renda', 'baixa', 'trapezoidal', (0, 0, 5, 15))
sistema.add_term('renda', 'media', 'trapezoidal', (5, 15, 20, 25))
sistema.add_term('renda', 'alta', 'trapezoidal', (20, 25, 30, 30))

# ============================================================================
# Variável 2: Escolaridade
# ============================================================================

sistema.add_input('escolaridade', (0, 15))

sistema.add_term('escolaridade', 'baixa', 'trapezoidal', (0, 0, 3, 6))
sistema.add_term('escolaridade', 'media', 'triangular', (4, 7.5, 11))
sistema.add_term('escolaridade', 'alta', 'trapezoidal', (9, 12, 15, 15))

# ============================================================================
# Saída: Chance de Votar (0 a 100%)
# ============================================================================

sistema.add_output('chance_voto', (0, 100))

sistema.add_term('chance_voto', 'baixa', 'triangular', (0, 0, 50))
sistema.add_term('chance_voto', 'media', 'triangular', (25, 50, 75))
sistema.add_term('chance_voto', 'alta', 'triangular', (50, 100, 100))

print('✅ Variáveis definidas!')
print('   • Renda: baixa, media, alta')
print('   • Escolaridade: baixa, media, alta')
print('   • Chance de voto: baixa, media, alta')

## Visualizar Funções de Pertinência

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(12, 12))

# Renda
x_renda = np.linspace(0, 30, 200)
for term_name, term in sistema.input_variables['renda'].terms.items():
    mu = term.membership(x_renda)
    axes[0].plot(x_renda, mu, linewidth=2, label=term_name.capitalize())
    axes[0].fill_between(x_renda, 0, mu, alpha=0.2)

axes[0].set_title('Renda Familiar (mil R$)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Renda (mil R$)')
axes[0].set_ylabel('Grau de Pertinência')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim([-0.05, 1.05])

# Escolaridade
x_esc = np.linspace(0, 15, 200)
for term_name, term in sistema.input_variables['escolaridade'].terms.items():
    mu = term.membership(x_esc)
    axes[1].plot(x_esc, mu, linewidth=2, label=term_name.capitalize())
    axes[1].fill_between(x_esc, 0, mu, alpha=0.2)

axes[1].set_title('Escolaridade (anos)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Anos de Estudo')
axes[1].set_ylabel('Grau de Pertinência')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim([-0.05, 1.05])

# Chance de Voto
x_chance = np.linspace(0, 100, 200)
for term_name, term in sistema.output_variables['chance_voto'].terms.items():
    mu = term.membership(x_chance)
    axes[2].plot(x_chance, mu, linewidth=2, label=term_name.capitalize())
    axes[2].fill_between(x_chance, 0, mu, alpha=0.2)

axes[2].set_title('Chance de Votar (%)', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Chance (%)')
axes[2].set_ylabel('Grau de Pertinência')
axes[2].legend()
axes[2].grid(True, alpha=0.3)
axes[2].set_ylim([-0.05, 1.05])

plt.tight_layout()
plt.show()

print('✅ Funções visualizadas!')

### Step 2

In [ ]:
# ============================================================================
# Definir Regras - BASEADAS EM PADRÕES ELEITORAIS REAIS
# ============================================================================

# Padrões observados em eleições:
# - Alta escolaridade + Qualquer renda → Maior probabilidade progressista
# - Baixa renda + Baixa escolaridade → Varia por região e perfil do candidato
# - Alta renda + Baixa escolaridade → Tende conservador
# - Classe média educada → Forte tendência progressista urbana

sistema.add_rules([
    # Alta escolaridade geralmente aumenta voto progressista
    {'renda': 'alta', 'escolaridade': 'alta', 'chance_voto': 'alta'},      # Classe alta educada urbana
    {'renda': 'media', 'escolaridade': 'alta', 'chance_voto': 'alta'},     # Classe média educada (base progressista forte)
    {'renda': 'baixa', 'escolaridade': 'alta', 'chance_voto': 'alta'},     # Baixa renda educada (consciência social)
    
    # Escolaridade média mostra padrão misto
    {'renda': 'alta', 'escolaridade': 'media', 'chance_voto': 'media'},    # Burguesia sem formação superior
    {'renda': 'media', 'escolaridade': 'media', 'chance_voto': 'media'},   # Classe média intermediária
    {'renda': 'baixa', 'escolaridade': 'media', 'chance_voto': 'media'},   # Trabalhadores com ensino médio
    
    # Baixa escolaridade tende a padrões mais conservadores ou populistas
    {'renda': 'alta', 'escolaridade': 'baixa', 'chance_voto': 'baixa'},    # Elite sem educação formal
    {'renda': 'media', 'escolaridade': 'baixa', 'chance_voto': 'baixa'},   # Pequenos empresários tradicionais
    {'renda': 'baixa', 'escolaridade': 'baixa', 'chance_voto': 'media'}    # Base popular (depende de pautas sociais)
])

# print('📜 Base de Regras (Baseada em Padrões Eleitorais Reais):')
# print('   R1: SE renda ALTA E escolaridade ALTA → chance ALTA (elite progressista urbana)')
# print('   R2: SE renda MÉDIA E escolaridade ALTA → chance ALTA (classe média educada)')
# print('   R3: SE renda BAIXA E escolaridade ALTA → chance ALTA (consciência social)')
# print('   R4: SE renda ALTA E escolaridade MÉDIA → chance MÉDIA (burguesia intermediária)')
# print('   R5: SE renda MÉDIA E escolaridade MÉDIA → chance MÉDIA (grupo swing)')
# print('   R6: SE renda BAIXA E escolaridade MÉDIA → chance MÉDIA (trabalhadores urbanos)')
# print('   R7: SE renda ALTA E escolaridade BAIXA → chance BAIXA (elite tradicional)')
# print('   R8: SE renda MÉDIA E escolaridade BAIXA → chance BAIXA (pequenos empresários)')
# print('   R9: SE renda BAIXA E escolaridade BAIXA → chance MÉDIA (voto clientelista/social)')

print(f'\n✅ {len(sistema.rule_base.rules)} regras criadas!')

sistema.plot_rule_matrix()


### Step 3

In [ ]:
# ============================================================================
# Caso de Teste: Renda=22, Escolaridade=12
# ============================================================================

print('='*70)
print('🧪 TESTE: Renda=22 mil, Escolaridade=12 anos')
print('='*70)

resultado = sistema.evaluate({
    'renda': 22,
    'escolaridade': 12
})

chance = resultado['chance_voto']

print(f'\n📊 Resultado: Chance de votar = {chance:.2f}%')

# Mostrar fuzzificação
graus_renda = sistema.input_variables['renda'].fuzzify(22)
graus_esc = sistema.input_variables['escolaridade'].fuzzify(12)

print(f'\nGraus de pertinência (Renda=22):')
for termo, mu in graus_renda.items():
    print(f'  {termo}: {mu:.3f}')

print(f'\nGraus de pertinência (Escolaridade=12):')
for termo, mu in graus_esc.items():
    print(f'  {termo}: {mu:.3f}')

print('='*70)

## Testar Múltiplos Cenários

In [ ]:
print('='*80)
print('🧪 TESTANDO MÚLTIPLOS CENÁRIOS')
print('='*80)
print()

cenarios = [
    (5, 3, 'Baixa renda, baixa escolaridade'),
    (15, 8, 'Renda média, escolaridade média'),
    (25, 12, 'Alta renda, alta escolaridade'),
    (10, 12, 'Renda média, alta escolaridade'),
    (28, 5, 'Alta renda, baixa escolaridade'),
]

print(f'{"Renda":<8} {"Escol":<8} {"Chance (%)":<12} {"Cenário"}')
print('-'*80)

for r, e, descricao in cenarios:
    resultado = sistema.evaluate({
        'renda': r,
        'escolaridade': e
    })
    c = resultado['chance_voto']
    print(f'{r:<8} {e:<8} {c:<12.2f} {descricao}')

print('\n' + '='*80)
print('✅ Testes concluídos!')

## Superfície de Controle 3D

In [ ]:
print('🔄 Gerando superfície de controle...')

# Criar grid
renda_range = np.linspace(0, 30, 30)
esc_range = np.linspace(0, 15, 30)
RENDA, ESC = np.meshgrid(renda_range, esc_range)

# Calcular chance para cada ponto
CHANCE = np.zeros_like(RENDA)
for i in range(RENDA.shape[0]):
    for j in range(RENDA.shape[1]):
        try:
            resultado = sistema.evaluate({
                'renda': RENDA[i,j],
                'escolaridade': ESC[i,j]
            })
            CHANCE[i,j] = resultado['chance_voto']
        except:
            CHANCE[i,j] = np.nan

# Plotar
fig = plt.figure(figsize=(16, 7))

ax1 = fig.add_subplot(121, projection='3d')
surf = ax1.plot_surface(RENDA, ESC, CHANCE, cmap='RdYlGn', alpha=0.9)
ax1.set_xlabel('Renda Familiar (mil R$)', fontsize=11)
ax1.set_ylabel('Escolaridade (anos)', fontsize=11)
ax1.set_zlabel('Chance de Votar (%)', fontsize=11)
ax1.set_title('Superfície de Controle 3D', fontsize=13, fontweight='bold')
ax1.view_init(elev=20, azim=135)
fig.colorbar(surf, ax=ax1, shrink=0.5, label='Chance (%)')

ax2 = fig.add_subplot(122)
contour = ax2.contourf(RENDA, ESC, CHANCE, levels=15, cmap='RdYlGn')
ax2.set_xlabel('Renda Familiar (mil R$)', fontsize=12)
ax2.set_ylabel('Escolaridade (anos)', fontsize=12)
ax2.set_title('Mapa de Contorno', fontsize=13, fontweight='bold')
fig.colorbar(contour, ax=ax2, label='Chance (%)')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print('✅ Superfície gerada!')
print('\n💡 Cores verdes = maior chance de votar')
print('   Cores vermelhas = menor chance de votar')